In [1]:
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers import models, losses
from sentence_transformers.util import cos_sim
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from glob import glob
import re
import pickle, gzip
import sys
from nltk.tokenize import sent_tokenize

sys.path.insert(0,'/home/sulcan/Documents/ipac-logbook/code/')
from mmd import *

i = 0

data_folder = '/home/sulcan/Documents/ipac-logbook/data/data_acc/'
model_folder = f'/home/sulcan/Documents/ipac-logbook/models/simcse/{i}'


folders = [f'{data_folder}/arxiv/',\
           f'{data_folder}/jacow/',\
           f'{data_folder}/books/']

files = []
for folder in folders:
    files.extend(glob(folder + '*.mmd'))

## Data Preparation

In [2]:
if False:
    # loading data
    print('loading data...')
    data_mmd = {}
    for file in tqdm(sorted(files)):
        with open(file, 'r') as f:
            data_mmd[file] = f.read()
    print('... data loaded')
    
    print('preparing data ...')
    # preparing equations and removing tables
    data_mmd = prepare_mmd_eqations_and_tables_for_simcse(data_mmd)
    print('... data prepared')
    
    print('chunking sentences ...')
    # chunking sentences
    train_sentences = []
    for k in tqdm(data_mmd):
        train_sentences.extend(sent_tokenize(data_mmd[k]))
    
    with open(f'{data_folder}/simcse_prepared_data.pickle.gzip','wb') as f:
        pickle.dump({'data_mmd' : data_mmd, 'train_sentences' : train_sentences}, f)
    print('...sentences chunked.')
    
else:
    '''
    with open(f'{data_folder}/simcse_prepared_data.pickle.gzip','rb') as f:
        data_mmd = pickle.load(f)
    '''
    with open(f'{data_folder}/simcse_prepared_data.pickle.gzip','rb') as f:
        data = pickle.load(f)# 
        data_mmd = data['data_mmd']
        train_sentences = data['train_sentences']

In [ ]:
# Define your sentence transformer model using CLS pooling
model_name = "allenai/scibert_scivocab_cased"
word_embedding_model = models.Transformer(model_name, max_seq_length=128)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model], device = 'cuda:1')

# Convert train sentences to sentence pairs
train_data = [InputExample(texts=[s, s]) for s in tqdm(train_sentences)]

# DataLoader to batch your data
train_dataloader = DataLoader(train_data, batch_size=192, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.MultipleNegativesRankingLoss(model)

# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)], epochs=5, show_progress_bar=True
)

# model.save("output/simcse-model")


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6231288/6231288 [00:21<00:00, 294874.24it/s]


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/32455 [00:00<?, ?it/s]

In [ ]:
if True:
    model.save(model_folder)

In [ ]:
e = model.encode(train_sentences)
cos_sim(e,e)

In [ ]:
model = None